# Retrieving Answers from MemDoc

Steps:

1. Create and Setup the Kernel with the appropriate model.
2. Acquire the data.
3. Load the data in Volatile Memory.
4. Create the prompts.
5. Ask questions and the get the result via using Recalling.

In [1]:
#r "nuget: Microsoft.SemanticKernel, 0.12.207.1-preview"

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.SemanticFunctions;
using Microsoft.SemanticKernel.KernelExtensions;
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.CoreSkills;

using System.Net.Http;

HttpClient httpClient = new();

Installed Packages Microsoft.SemanticKernel, 0.12.207.1-preview

## Kernel Setup

In [2]:
string apiKey = "";
string model = "text-davinci-003";
string orgId = "";

In [4]:
using Microsoft.SemanticKernel.Memory;

var kernel = new KernelBuilder()
    .Configure(c =>
    {
        /* To Use: AzureOpenAI
        if (useAzureOpenAI)
        {
            c.AddAzureTextEmbeddingGenerationService("ada", "text-embedding-ada-002", azureEndpoint, apiKey);
            c.AddAzureTextCompletionService("davinci", model, azureEndpoint, apiKey);
        }
        */
        c.AddOpenAITextEmbeddingGenerationService("ada", "text-embedding-ada-002", apiKey);
        c.AddOpenAITextCompletionService("text-davinci-003", model, apiKey, orgId);
    })
    .WithMemoryStorage(new VolatileMemoryStore())
    .Build();

## Data Acquisition

In [5]:
// Extract Mem Doc questions
string url = "https://raw.githubusercontent.com/dotnet/coreclr/master/Documentation/botr/garbage-collection.md";
var response = await httpClient.GetAsync(url);
string content = await response.Content.ReadAsStringAsync();
string[] split = content.Split("##", StringSplitOptions.RemoveEmptyEntries | StringSplitOptions.TrimEntries);

## Load Up The Data in a Memory

In [7]:
const string MemoryCollectionName = "memdocQnA";

var memorySkill = new TextMemorySkill();
kernel.ImportSkill(memorySkill);

// Build a semantic function that saves info to memory
const string SaveFunctionDefinition = @"{{save $info}}";
var memorySaver = kernel.CreateSemanticFunction(SaveFunctionDefinition);

var context = kernel.CreateNewContext();
context[TextMemorySkill.CollectionParam] = MemoryCollectionName;

for (int i = 0; i < split.Length; i++)
{
    context[TextMemorySkill.KeyParam] = $"q{i}";
    context["info"] = split[i];
    await memorySaver.InvokeAsync(context);
}

/*
// Memory can be stored alternatively like this:

for (int i = 0; i < split.Length - 210; i++)
{
    await kernel.Memory.SaveInformationAsync(MemoryCollectionName, id: $"q{i}", text: split[i]);
}
*/

## QnA

In [25]:
// Set up the Semantic Function.
const string skPrompt = @"
Answer the question below with as much depth and detail as possible considering only the information delimited by ===================
.
If the question is irrelevant to the topics above, reply 'I don't know'.
GC in this context is always the Garbage Collector or Garbage Collection and no other term.

=================
{{recall $input}}
=================

Question: {{$input}}

Answer:";
context[TextMemorySkill.LimitParam] = "1";
context[TextMemorySkill.RelevanceParam] = "0.9";

var chatFunction = kernel.CreateSemanticFunction(skPrompt, maxTokens: 1000, temperature: 0.0);
Func<string, Task> Chat = async (string query) =>
{
    context["input"] = query;
    var res = await chatFunction.InvokeAsync(context);
    Console.WriteLine($"{query}\n >> {res}");
};

### Generic Questions

In [15]:
await Chat("What's the role of the mark phase? Explain in depth.");

What's the role of the mark phase? Explain in depth.
 >>  The mark phase is the second phase of the garbage collection process. It is responsible for identifying which objects in memory are still in use and which are no longer needed. It does this by traversing the object graph and marking each object it finds as either in use or no longer needed. This allows the garbage collector to determine which objects can be safely removed from memory.


In [18]:
await Chat("What are the GC phases?")

What are the GC phases?
 >>  The GC phases are: SuspendEE, Mark, Plan, Relocate and Compact (if necessary), Sweep (if necessary), and RestartEE.


In [26]:
await Chat("What are the goals of the GC?")

What are the goals of the GC?
 >>  The main goal of the Garbage Collector (GC) is to reclaim memory that is no longer being used by a program. This is done by identifying objects that are no longer referenced by the program and then freeing up the memory associated with those objects. The GC also helps to improve the performance of a program by reducing the amount of memory that needs to be allocated and managed by the program. Additionally, the GC can help to reduce the amount of time spent on memory management tasks, freeing up resources for other tasks.


## Debugging

In [1]:
using System.Diagnostics;

Console.WriteLine($"Current Process ID: {System.Diagnostics.Process.GetCurrentProcess().Id}");

#!about

Current Process ID: 27312


.NET Interactive© 2020 Microsoft CorporationVersion: 1.0.425803+1db2979099d0272660e1497cae9b9af1238db42fLibrary version: 1.0.0-beta.23258.3+1db2979099d0272660e1497cae9b9af1238db42fBuild date: 2023-05-12T10:30:52.4965699Zhttps://github.com/dotnet/interactive
